In [1]:
import skimage
import os
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO
import requests
import keras

from keras.models import load_model
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Activation, LeakyReLU, Flatten, Reshape
from keras.models import Model, Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
import pickle
import numpy as np

%matplotlib inline

Using TensorFlow backend.


In [2]:
images = ['../../data/posters/10547_poster.jpg', '../../data/posters/10312_poster.jpg']

In [3]:
autoencoder = Sequential()

# autoencoder.add(Input(shape=(32, 512, 512, 3)))
autoencoder.add(Conv2D(64, (7, 7), padding='same', input_shape=(512, 512, 3)))  # first conv layer
autoencoder.add(BatchNormalization())
autoencoder.add(LeakyReLU(alpha=0.1))
autoencoder.add(MaxPooling2D((2, 2), padding='same'))
autoencoder.add(Conv2D(128, (5, 5), padding='same'))  # second conv layer
autoencoder.add(BatchNormalization())
autoencoder.add(LeakyReLU(alpha=0.1))
autoencoder.add(MaxPooling2D((2, 2), padding='same'))
autoencoder.add(Conv2D(256, (3, 3), padding='same'))  # third conv layer
autoencoder.add(BatchNormalization())
autoencoder.add(LeakyReLU(alpha=0.1))
autoencoder.add(MaxPooling2D((2, 2), padding='same'))
autoencoder.add(Conv2D(512, (3, 3), padding='same'))  # fourth conv layer
autoencoder.add(BatchNormalization())
autoencoder.add(LeakyReLU(alpha=0.1))
autoencoder.add(MaxPooling2D((2, 2), padding='same'))
autoencoder.add(Conv2D(512, (3, 3), padding='same'))  # fifth conv layer
autoencoder.add(BatchNormalization())
autoencoder.add(LeakyReLU(alpha=0.1))
autoencoder.add(MaxPooling2D((2, 2), padding='same'))
autoencoder.add(Flatten())  # resulting size: (None, 16*16*1024)
autoencoder.add(Dense(1024, activation='relu'))
autoencoder.add(Dense(16*16*512, activation='relu'))
autoencoder.add(Reshape((16, 16, 512)))
autoencoder.add(UpSampling2D((2, 2)))
autoencoder.add(Conv2D(512, (3, 3), padding='same'))  # -fifth conv layer
autoencoder.add(BatchNormalization())
autoencoder.add(LeakyReLU(alpha=0.1))
autoencoder.add(UpSampling2D((2, 2)))
autoencoder.add(Conv2D(512, (3, 3), padding='same'))  # -fourth conv layer
autoencoder.add(BatchNormalization())
autoencoder.add(LeakyReLU(alpha=0.1))
autoencoder.add(UpSampling2D((2, 2)))
autoencoder.add(Conv2D(256, (3, 3), padding='same'))  # -third conv layer
autoencoder.add(BatchNormalization())
autoencoder.add(LeakyReLU(alpha=0.1))
autoencoder.add(UpSampling2D((2, 2)))
autoencoder.add(Conv2D(128, (5, 5), padding='same'))  # -second conv layer
autoencoder.add(BatchNormalization())
autoencoder.add(LeakyReLU(alpha=0.1))
autoencoder.add(UpSampling2D((2, 2)))
autoencoder.add(Conv2D(64, (7, 7), padding='same'))  # -first conv layer
autoencoder.add(BatchNormalization())
autoencoder.add(LeakyReLU(alpha=0.1))
autoencoder.add(Conv2D(3, (1, 1), padding='same', activation='relu'))  # 1x1 conv to reduce channels to bring into input image shape
# might need a reshape layer for enabling image comparison

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 512, 512, 64)      9472      
_________________________________________________________________
batch_normalization_1 (Batch (None, 512, 512, 64)      256       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512, 512, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 256, 256, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 256, 256, 128)     204928    
_________________________________________________________________
batch_normalization_2 (Batch (None, 256, 256, 128)     512       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256, 256, 128)     0         
__________